In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [2]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_10Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [3]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [4]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [5]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [6]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [7]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
#     prompts = read_payloads(request_payload_file)
    
    global df
    for i in range(10):
        prompts = read_payloads(request_payload_file)
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-69f5b40a59f244b28872ebb760f2853f","object":"text_completion","created":445905,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 3.298591 seconds 


Prompt==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-8276121dd5254ccdb1a1b0f6ca328d5c","object":"text_completion","created":445905,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/c

Prompt==> Problem: Write a function in python to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-8a9bc48cbfc34955a53f706102d9c8fc","object":"text_completion","created":445905,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ...\n\nThe Fibonacci sequence is defined as follows:\n\nF(0) = 0, F(1) = 1\n\nFor n > 1, F(n) = F(n-1) + F(n-2)\n\nHere is a Python function that generates the nth Fibonacci number:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        return \"Input should be a positive integer.\"\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        a, b = 0, 1\n       ","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,"total_tokens":243,"completion_tokens":200}}
Execution time: 11.893238 

Prompt==> Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-208dabb1fa704a4fabe742ce391d9a0f","object":"text_completion","created":445915,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```java\npublic class BinarySearch {\n    public static int binarySearch(int[] arr, int target) {\n        int left = 0;\n        int right = arr.length - 1;\n\n        while (left <= right) {\n            int mid = left + (right - left) / 2;\n\n            if (arr[mid] == target) {\n                return mid;\n            } else if (arr[mid] < target) {\n                left = mid + 1;\n            } else {\n                right = mid - 1;\n            }\n        }\n\n        return -1;\n    }\n\n    public static void main(String[] args) {\n        int[

Prompt==> Problem: Write a function in javascript to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-eedbf22037284512b39f5eb1cc5e8c4c","object":"text_completion","created":445931,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction isPalindrome(s) {\n    let reversed = s.split('').reverse().join('');\n    return s === reversed;\n}\n\nconsole.log(isPalindrome('madam')); // true\nconsole.log(isPalindrome('hello')); // false","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":39,"total_tokens":109,"completion_tokens":70}}
Execution time: 3.970810 seconds 


Prompt==> Problem: Write a function in c++ to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-052082c93bfd4090b11512a4e99d4671","object":"text_completion","crea

Prompt==> Problem: Write a function in javascript to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime. 
 {"id":"cmpl-4de3ee4482654b8e97d0fbeb36a1068d","object":"text_completion","created":445930,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction isPrime(num) {\n    if (num <= 1) {\n        return false;\n    }\n    if (num === 2) {\n        return true;\n    }\n    if (num % 2 === 0) {\n        return false;\n    }\n    for (let i = 3; i <= Math.sqrt(num); i += 2) {\n        if (num % i === 0) {\n            return false;\n        }\n    }\n    return true;\n}\n\nconsole.log(isPrime(2)); // true\nconsole.log(isPrime(4)); // false\nconsole.log(isPrime(7)); // true\nconsole.log(isPrime(10)); // false\nconsole.log(isPrime(13)); // true\nconsole.log(isPrime(15)); // false","logprobs":null,"finish_reason":"stop"}],"usag

...

Prompt==> Problem: Implement an autocomplete feature in jQuery for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions). 
 {"id":"cmpl-6d52e4f002ac4028811b3524aa8177c3","object":"text_completion","created":446199,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple implementation of autocomplete feature using jQuery UI's Autocomplete widget.\n\nHTML:\n```html\n<input id=\"search\" type=\"text\">\n```\n\nJavaScript:\n```javascript\nvar data = [\"Apple\", \"Banana\", \"Cherry\", \"Date\", \"Elderberry\", \"Fig\", \"Grape\", \"Honeydew\", \"Iceberg Lettuce\", \"Jackfruit\", \"Kiwi\", \"Lemon\", \"Mango\", \"Nectarine\", \"Orange\", \"Papaya\", \"Quince\", \"Raspberry\", \"Strawberry\", \"Tangerine\", \"Ugli Fruit\", \"Vanilla\", \"Watermelon\", \"Xigua\", \"Yuzu\", \"Zucchini\"];\n\n$(function() {\n    $(\"#search","

Prompt==> Problem: Build a function in jQuery to populate an HTML table with data from an array of objects.Input: Array of objects, each representing a row in the table.Output: None (update the HTML table). 
 {"id":"cmpl-9c6c14819a2d41628b21a68b595f6f0b","object":"text_completion","created":446205,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can do this:\n\n```javascript\nfunction populateTable(data) {\n    var table = $('<table/>');\n    $.each(data, function(i, item) {\n        var row = $('<tr/>');\n        $.each(item, function(key, value) {\n            var cell = $('<td/>').text(value);\n            row.append(cell);\n        });\n        table.append(row);\n    });\n    $('body').append(table);\n}\n```\n\nIn this function, we first create a new table element. Then, for each item in the data array, we create a new row and append it to the table.

Prompt==> Problem: Write a function in java to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-a604a388fa6a40faad368442fbbd57cc","object":"text_completion","created":446222,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\npublic class Main {\n    public static void main(String[] args) {\n        String s = \"Hello World\";\n        System.out.println(reverseString(s));\n    }\n\n    public static String reverseString(String s) {\n        String reversed = \"\";\n        for (int i = s.length() - 1; i >= 0; i--) {\n            reversed += s.charAt(i);\n        }\n        return reversed;\n    }\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":146,"completion_tokens":115}}
Execution time: 6.552099 seconds 


Prompt==> Problem: Write a function in python to perform binary search on a sorted list.Input: Sorte

Prompt==> Problem: Write a function in java to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-741e66f52cde41d4a7970f29ed42bd97","object":"text_completion","created":446227,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, …\n\nHere is a simple Java function to generate the nth Fibonacci number:\n\n```java\npublic class Main {\n    public static void main(String[] args) {\n        System.out.println(fibonacci(10));  // Output: 55\n    }\n\n    public static int fibonacci(int n) {\n        if (n <= 1) {","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,"total_tokens":243,"completion_tokens":200}}
Execution time: 11.477590 seconds 


Prompt==> Problem: Write a function in java to count t

Prompt==> Problem: Write a function in javascript to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-7783c3a6f9ed492892de55fe5c673c97","object":"text_completion","created":446242,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction factorial(n) {\n    if (n === 0) {\n        return 1;\n    } else {\n        return n * factorial(n - 1);\n    }\n}\n\nconsole.log(factorial(5)); // Output: 120","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":40,"total_tokens":107,"completion_tokens":67}}
Execution time: 3.882754 seconds 


Prompt==> Problem: Write a function in c++ to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-adc8385dfada49369ba2c109e951baef","object":"text_completion","created":446235,"model":"/llmmod

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))